# 第三章 构建项目数据库

在前面两章我们具体介绍了我们的知识库选择，并介绍了如何加载、处理数据并构建、使用向量数据库。在本章中，我们结合之前讲解的内容，具体构建出本项目使用的数据库，之后我们的 Prompt Engineeering 及迭代优化、验证评估均会在该数据库基础上进行。

在这里，我们使用 OpenAI 的 Embedding 模型实现向量化，大家也可以根据前面的讲解选用不同的 Embedding 模型。

由于此处加载向量数据库需要一定时间运行，我们也提供了 py 脚本供大家使用。

In [6]:
# 首先实现基本配置
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import UnstructuredFileLoader

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline

# 使用前配置自己的 api 到环境变量中如
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env fileopenai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key  = os.environ['OPENAI_API_KEY']

接下来逐个加载知识库里的文档：

In [2]:
#pdf
# 加载 PDF
loaders = [
    PyMuPDFLoader("../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf") # 机器学习,
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [5]:
#md
folder_path = "../../data_base/knowledge_db/prompt_engineering/"
files = os.listdir(folder_path)
loaders = []
for one_file in files:
    loader = UnstructuredMarkdownLoader(os.path.join(folder_path, one_file))
    loaders.append(loader)
for loader in loaders:
    docs.extend(loader.load())

In [7]:
#mp4-txt
loaders = [
    UnstructuredFileLoader("../../data_base/knowledge_db/easy_rl/强化学习入门指南.txt") # 机器学习,
]
for loader in loaders:
    docs.extend(loader.load())

然后对加载好的文档进行切片并向量化后存储到向量数据库中：

In [13]:
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)


# 定义 Embeddings
embedding = OpenAIEmbeddings() 

# 定义持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

Using embedded DuckDB with persistence: data will be stored in: ../../data_base/vector_db/chroma


最后将加载好的向量数据库持久化即可：

In [ ]:
vectordb.persist()